<a href="https://colab.research.google.com/github/Bharathjpv/Red-wine-quality/blob/main/Project_red_wine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import pandas as pd
data = pd.read_csv('winequality-red.csv', sep=';')
data

In [ ]:
data.value_counts(data['quality'])

In [ ]:
data.isnull().sum(axis=0)

In [ ]:
data['quality'].hist()

Saparate X and Y

In [ ]:
X = data.iloc[:, :-1]
Y = data.iloc[:, -1]

In [ ]:
from sklearn.preprocessing import StandardScaler
scalar_ = StandardScaler()

#X = scalar_.fit_transform(X)

In [ ]:
X = pd.DataFrame(X, columns=['fixed acidity','volatile acidity',	'citric acid',	'residual sugar',	'chlorides',	'free sulfur dioxide',	'total sulfur dioxide',	'density',	'pH',	'sulphates',	'alcohol'])
X

In [ ]:
data.corr()

Drop Fixed acidity and free sulphur dioxide

In [ ]:
X.drop(columns=['fixed acidity', 'free sulfur dioxide'], inplace=True)

Train test split of the data

In [ ]:
 from sklearn.model_selection import train_test_split
 X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42, stratify=Y)

scaling training and testing data separately

In [ ]:
X_train = scalar_.fit_transform(X_train)
X_test = scalar_.fit_transform(X_test)

Over sampling of the train data

In [ ]:
from imblearn.combine import SMOTETomek
smk = SMOTETomek(random_state=42)
X_train, Y_train = smk.fit_sample(X_train, Y_train)

Convert data from numpy array to pandas dataframe

In [ ]:
Y_train = pd.DataFrame(Y_train, columns=['quality'])
Y_train.value_counts()

quality
8          477
3          477
7          476
4          476
6          470
5          470
dtype: int64

In [ ]:
X_train = pd.DataFrame(X_train, columns=['volatile acidity',	'citric acid',	'residual sugar',	'chlorides','total sulfur dioxide',	'density',	'pH',	'sulphates',	'alcohol'])
X_train

,volatile acidity,citric acid,residual sugar,chlorides,total sulfur dioxide,density,pH,sulphates,alcohol
0,-0.763972,-0.696014,-0.177840,-0.731583,-0.375344,-1.509535,1.882722,0.047813,1.380643
1,-0.743505,0.102021,-0.584156,-0.166832,0.985048,0.307744,0.767702,-0.463380,-0.992450
2,-0.661639,1.099565,1.609948,-0.295185,0.161653,0.673288,-2.052643,-0.335582,-0.992450
3,0.402620,-1.244663,-0.665419,0.064202,1.987441,-1.102214,1.358006,-0.910674,-0.583296
4,-0.456974,0.002267,-0.259104,-0.757253,-0.375344,-0.705337,-1.003213,0.878502,0.398673
...,...,...,...,...,...,...,...,...,...
2841,-0.109761,-0.064010,-0.277704,0.017310,0.589289,1.462041,-0.902060,-0.548288,-0.494549
2842,0.338453,-1.039384,-0.155412,0.257729,0.038270,0.310467,-0.486085,0.049671,-0.753580
2843,0.320754,0.600793,0.472265,-0.038480,0.734449,1.247715,0.242986,0.303410,0.644165
2844,-1.018077,0.377717,-0.114993,-0.437597,0.006543,-0.773220,0.046850,0.706459,1.216192


We can't try logistric regression since its not a binoial response.
Apply SVM, Decision tree and random forest classiffier.

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix

In [ ]:
svc = SVC(kernel='rbf', random_state=42)
svc.fit(X_train, Y_train)
Y_predict_svc = svc.predict(X_test)

In [ ]:
cmsvc = confusion_matrix(Y_test, Y_predict_svc)
cmsvc

array([[  1,   1,   1,   0,   0,   0],
       [  1,   9,   4,   2,   0,   0],
       [  9,  36, 117,  38,   2,   2],
       [  5,  21,  40,  90,  26,  10],
       [  0,   1,   3,  13,  28,  15],
       [  0,   0,   0,   0,   2,   3]])

In [ ]:
scoresvc = svc.score(X_test, Y_test)
scoresvc

0.5166666666666667

51 percent accuracy is not a great model

In [ ]:
from sklearn.tree import DecisionTreeClassifier
dtc = DecisionTreeClassifier(random_state=42)

In [ ]:
dtc.fit(X_train, Y_train)
Y_predict_dtc = dtc.predict(X_test)

In [ ]:
cmdtc = confusion_matrix(Y_test, Y_predict_dtc)
cmdtc

array([[  0,   1,   2,   0,   0,   0],
       [  2,   7,   6,   1,   0,   0],
       [  7,  14, 120,  54,   9,   0],
       [  0,  11,  54,  96,  23,   8],
       [  0,   0,   7,  19,  25,   9],
       [  0,   0,   0,   1,   2,   2]])

In [ ]:
score_dtc = dtc.score(X_test, Y_test)
score_dtc

0.5208333333333334

52 is not good score. So continu with the next classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(random_state=42)
rfc.fit(X_train, Y_train)
Y_predict_rfc = rfc.predict(X_test)

In [ ]:
cm_rfc = confusion_matrix(Y_test, Y_predict_rfc)
cm_rfc

array([[  0,   1,   2,   0,   0,   0],
       [  2,   6,   4,   4,   0,   0],
       [  4,   8, 142,  46,   4,   0],
       [  1,   6,  48, 109,  20,   8],
       [  0,   0,   3,  24,  27,   6],
       [  0,   0,   0,   1,   3,   1]])

In [ ]:
score_rfc = rfc.score(X_test, Y_test)
score_rfc

0.59375

62 is a good score. Now we shall evaluate the models bult and test their accuray score on train and test dataset

In [ ]:
from sklearn.metrics import classification_report
cr_svm = classification_report(Y_test, Y_predict_svc)
cr_dtc = classification_report(Y_test, Y_predict_dtc)
cr_rfc = classification_report(Y_test, Y_predict_rfc)

In [ ]:
print(cr_svm),print(cr_dtc), print(cr_rfc)

              precision    recall  f1-score   support

           3       0.06      0.33      0.11         3
           4       0.13      0.56      0.21        16
           5       0.71      0.57      0.63       204
           6       0.63      0.47      0.54       192
           7       0.48      0.47      0.47        60
           8       0.10      0.60      0.17         5

    accuracy                           0.52       480
   macro avg       0.35      0.50      0.36       480
weighted avg       0.62      0.52      0.55       480

              precision    recall  f1-score   support

           3       0.00      0.00      0.00         3
           4       0.21      0.44      0.29        16
           5       0.63      0.59      0.61       204
           6       0.56      0.50      0.53       192
           7       0.42      0.42      0.42        60
           8       0.11      0.40      0.17         5

    accuracy                           0.52       480
   macro avg       0.32

(None, None, None)

Cross validation of the data

In [ ]:
from sklearn.model_selection import cross_validate

In [ ]:
import numpy as np

In [ ]:
svc_cv = cross_validate(svc, X_train, Y_train, cv=10, return_train_score=True)
dtc_cv = cross_validate(dtc, X_train, Y_train, cv=10, return_train_score=True)
rfc_cv = cross_validate(rfc, X_train, Y_train, cv=10, return_train_score=True)

In [ ]:
avg_svc_test = np.average(svc_cv['test_score'])
avg_dtc_test = np.average(dtc_cv['test_score'])
avg_rfc_test = np.average(rfc_cv['test_score'])

In [ ]:
avg_svc_train = np.average(svc_cv['train_score'])
avg_dtc_train = np.average(dtc_cv['train_score'])
avg_rfc_train = np.average(rfc_cv['train_score'])

In [ ]:
print(f'{avg_svc_test}')
print(f'{avg_svc_train}')
print()
print(f'{avg_dtc_test}')
print(f'{avg_dtc_train}')
print()
print(f'{avg_rfc_test}')
print(f'{avg_rfc_train}')

0.7625435227854582
0.7954870976653885

0.8123259088581669
1.0

0.8932629288274448
1.0


# Hyperparametric tuning of the data

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
params_svc = {'C': [1, 10],
              'kernel': ['rbf'],
              'gamma' : [0.5, 1]
              }

grid_svc = GridSearchCV(svc, params_svc, scoring='accuracy', cv=10, n_jobs=1, return_train_score=True, verbose=1)
grid_svc_fit = grid_svc.fit(X_train, Y_train)
cv_results_svc = pd.DataFrame.from_dict(grid_svc_fit.cv_results_)
cv_results_svc.loc[(cv_results_svc['rank_test_score'] == 1)][['mean_test_score', 'params', 'mean_fit_time', 'mean_train_score']]

Fitting 10 folds for each of 4 candidates, totalling 40 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  40 out of  40 | elapsed:   16.1s finished


,mean_test_score,params,mean_fit_time,mean_train_score
3,0.909729,"{'C': 10, 'gamma': 1, 'kernel': 'rbf'}",0.29849,0.997345


2	0.887542	{'C': 1, 'gamma': 2, 'kernel': 'rbf'}
2	0.896142	{'C': 5, 'gamma': 1, 'kernel': 'rbf'}
6	0.897574	{'C': 10, 'gamma': 1, 'kernel': 'rbf'}
3	0.897932	{'C': 25, 'gamma': 1, 'kernel': 'rbf'}
1	0.820926	{'C': 100, 'kernel': 'poly'}

In [ ]:
cv_results_svc

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_gamma,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
0,0.244276,0.012259,0.029649,0.001324,25,1,rbf,"{'C': 25, 'gamma': 1, 'kernel': 'rbf'}",0.88551,0.883721,0.901434,0.897849,0.921147,0.897932,0.013472,1,0.998209,0.998209,0.998209,0.998209,0.998209,0.998209,3.928201e-07


In [ ]:
param_dtc = {'max_depth':[80],
             'min_samples_split':[1, 2],
             'max_features': [9, 15, 50]
             }
grid_dtc = GridSearchCV(dtc, param_dtc,scoring='accuracy', cv=5, n_jobs=1, return_train_score=True, verbose=1)
grid_dtc_fit = grid_dtc.fit(X_train, Y_train)

Fitting 5 folds for each of 6 candidates, totalling 30 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:    0.2s finished


In [ ]:
cv_results_dtc = pd.DataFrame.from_dict(grid_dtc_fit.cv_results_)
cv_results_dtc.loc[(cv_results_dtc['rank_test_score'] == 1)][['mean_test_score', 'mean_train_score', 'param_max_depth', 'param_max_features',	'param_min_samples_split']]

,mean_test_score,mean_train_score,param_max_depth,param_max_features,param_min_samples_split
1,0.803379,1.0,80,9,2


mean_test_score	param_max_depth	param_max_features	param_min_samples_split
32	0.796575	12	9	2
1	0.803379	20	9	2

In [ ]:
param_rfc = {'n_estimators':[80], 
            'min_samples_split':[3],
            'min_samples_leaf':[1]
            }
grid_rfc = GridSearchCV(rfc, param_rfc,scoring='accuracy', cv=10, n_jobs=1, return_train_score=True, verbose=1)
grid_rfc_fit = grid_rfc.fit(X_train, Y_train)

Fitting 10 folds for each of 1 candidates, totalling 10 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    5.2s finished


In [ ]:
cv_results_rfc = pd.DataFrame.from_dict(grid_rfc_fit.cv_results_)
cv_results_rfc.loc[(cv_results_rfc['rank_test_score'] == 1)][['mean_test_score', 'mean_train_score', 'param_n_estimators', 'param_min_samples_split', 'param_min_samples_leaf', 'mean_fit_time']]

,mean_test_score,mean_train_score,param_n_estimators,param_min_samples_split,param_min_samples_leaf,mean_fit_time
0,0.893976,1.0,80,3,1,0.473669



mean_test_score	param_n_estimators	param_min_samples_split	param_min_samples_leaf
4	0.893976	80	3	1

In [ ]:
svc_test = SVC(C=10, kernel='rbf', gamma=1, random_state=42)
svc_test.fit(X_train, Y_train)
Y_pred_test_svc = svc_test.predict(X_test)

In [ ]:
cm_test_svc = confusion_matrix(Y_test, Y_pred_test_svc)
cm_test_svc

array([[  0,   1,   2,   0,   0,   0],
       [  0,   5,   6,   5,   0,   0],
       [  0,   3, 145,  51,   5,   0],
       [  0,   5,  64, 112,   9,   2],
       [  0,   1,   9,  24,  21,   5],
       [  0,   0,   2,   1,   1,   1]])

In [ ]:
from sklearn.metrics import accuracy_score

score_test_svc = accuracy_score(Y_test, Y_pred_test_svc)
score_test_svc

0.5916666666666667

In [ ]:
rfc_test = RandomForestClassifier(n_estimators=80, min_samples_split=3, min_samples_leaf=1)
rfc_test.fit(X_train, Y_train)
Y_pred_test_rfc = rfc_test.predict(X_test)

In [ ]:
score_test_rfc = accuracy_score(Y_test, Y_pred_test_rfc)
score_test_rfc

0.5958333333333333